# 1/29/21 - This notebook selects and prepares a first LinKS candidate for modeling.

In [ ]:
### libraries
%matplotlib inline

from autoconf import conf
import autolens as al
import autolens.plot as aplt
import autofit as af
import pandas as pd
import numpy as np
from astropy.io import fits

# paths
autoz_path = '/data/sknabel/autoz_lens_model/'
file_path = f'{autoz_path}files/'
csv_path = f'{file_path}csv/'
fits_path = f'{file_path}fits/'
png_path = f'{autoz_path}visuals/png/'
pdf_path = f'{autoz_path}visuals/pdf/'

In [3]:
# load links
links = pd.read_csv(f'{csv_path}latest/links_sample_latest.csv')
print(len(links), links.columns)

39 Index(['Unnamed: 0', 'Unnamed: 0_x', 'GAMA_ID', 'ID', 'RA', 'DEC', 'score',
       'Unnamed: 0_y', 'z1', 'sigma1', 'template1', 'z2', 'sigma2',
       'template2', 'sigma3', 'sigma4', 'R', 'prob', 'error', 'type',
       'type_number', 'lambdar_log_mstar', 'lambdar_mstar',
       'lambdar_log_mstar_error', 'z', 'zlens', 'zsource', 'sigma_lens',
       'sigma_source', 'dlens', 'dsource', 'corrected_stellar_mass', 'P1',
       'P3', 'normalized_score', 'cnn_probability', 'm_half_re', 'sigma_star',
       'theta_e_pm', 'theta_e_sis'],
      dtype='object')


In [13]:
links.sort_values(by='score', ascending=False).head()[['GAMA_ID',
                                                      'ID',
                                                      'score',
                                                      'zlens',
                                                      'zsource',
                                                      'corrected_stellar_mass',
                                                      'theta_e_pm',
                                                     'theta_e_sis']]

,GAMA_ID,ID,score,zlens,zsource,corrected_stellar_mass,theta_e_pm,theta_e_sis
18,323152.0,2967,30.0,0.353286,0.721590,NaN,NaN,NaN
17,250289.0,2730,28.0,0.401214,0.719616,4.664644e+11,0.917868,0.645859
19,3575500.0,1906,26.0,0.232537,0.365195,3.049251e+11,1.296096,0.762649
31,3629152.0,1933,26.0,0.406623,0.787118,4.270558e+11,0.896116,0.659574
11,544226.0,3212,18.0,0.227044,0.649824,1.928528e+11,0.875230,0.725515


## I am selecting G3575500 (1906) with score of	26, redshifts 0.232537 and 0.365195, mass 3.049251e+11, and $\theta_E$	1.296096 and 0.762649

In [14]:
def get_noisey(folder, file): # insert fits file path (f'{fits_image_path}links/G262874_26/G262874_26_r.fits')
    
    # get the file ready
    hdul = fits.open(f'{folder}{file}') # open the fits
    print(f'Gettin noisey with {hdul[0].header[6]}') # list object name
    print(hdul.info())
    image = hdul[0].data # select first HDU to be the image
    
    # plot image data
    plt.figure()
    plt.title = (f'{hdul[0].header[6]}')
    plt.imshow(image)#, cmap='gray') # show image in grayscale
    plt.show()
    #
    plt.figure()
    plt.title = (f'{hdul[0].header[6]} signal')
    plt.hist(np.ravel(image),bins=100,log=True) # plot signal in histogram
    plt.show()
    print(f'Mean {np.mean(image)}, Median {np.median(image)}')
    
    # clip to 3sigma around median
    print('Clipping to 3-sigma about median value')
    clipped_image=clip(image, cenfunc='median')
    plt.figure()
    plt.title=(f'{hdul[0].header[6]} noise')
    plt.hist(np.ravel(clipped_image),bins=100,log=True) # plot signal in histogram
    plt.show()
    
    # generate Gaussian noise about mean and std
    mean = clipped_image.mean()
    std = clipped_image.std()
    noise_map = np.random.normal(loc=mean,
                                scale=std,
                                size= image.shape
                                )
    
    # plot noise map
    plt.figure() # show histogram of signal values
    plt.hist(np.ravel(noise_map))
    plt.show()
    plt.figure() # show noise map image
    plt.title=(f'Noise Map for {hdul[0].header[6]}')
    plt.imshow(noise_map, cmap='gray')
    
    # write to fits file
    hdu = fits.PrimaryHDU(noise_map)
    hdu.writeto(f'{folder}{file[:-5]}_noisemap.fits')
    print(f'Image sent to {folder}{file[:-5]}_noisemap.fits')
    
    # good vibes
    print('Congratulations! You simulated random noise! Eat a cookie.')
    
    
#get_noisey(f'{fits_image_path}links/G262874_26/G262874_26_r.fits') # it works!

In [16]:
folder = f'{fits_path}G3575500_1906/'
file = '1906_r.fits'

get_noisey(folder, file)

FileNotFoundError: [Errno 2] No such file or directory: '/data/sknabel/autoz_lens_model/files/fits/G3575500_1906/1906_r.fits'